In [1]:
import os
from datetime import datetime
import concurrent
import pandas_datareader.data as web
import pandas as pd
import datetime
import concurrent.futures
from concurrent.futures import wait, ALL_COMPLETED
import yfinance as yf
import urllib
import urllib.request
import time
from datetime import timedelta

from ipywidgets import interactive
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

#frequency = output_variable.value
frequency = "Q"

w=117*8
#start = datetime.datetime(2010, 1, 1)

#end = datetime.datetime(2013, 1, 27)
end_date = datetime.date.today()

start_date = end_date - timedelta(weeks=w)

pd.set_option('display.max_columns', None) #replace n with the number of columns you want to see completely
pd.set_option('display.max_rows', None) #replace n with the number of rows you want to see completely

cores = int(len(os.sched_getaffinity(0)))

print(end_date)
print(start_date)


etf_commodities = ['DBO','CORN', 'WEAT', 'SOYB', 'JO', 'SGG', 'BAL', 'COW', 'MOO', 'TAGS', 'KOL' ]
#Gold, Silver, Platinum, Copper, Paladium, Aluminum, Iron, Steel
etf_metals = ['IAU', 'SLV', 'PGM', 'JJC', 'PALL', 'JJU', 'IFUNX', 'SLX']
#US dollar, European Euro, Japanese yen, Pound sterling, Australian dollar, Canadian dollar, Swiss franc, Chinese Yuan Renminbi, Swedish Krona, Peso, India
#defunct: Russia: XRU, Mexico: FXM
etf_foreign_exchanges = ['UUP','FXE','FXY','FXB','FXA','FXC','FXF','CYB', 'FXS', 'INR']
#residential, Ishares all NAmerica
etf_real_estate = ['REZ', 'IYR']
#Russia, Germany, UK, Japan, China, Euro, Euro, Brazil, Latin America, Mexico, India
etf_economies = ['ERUS','EWG','EWU','EWJ','MCHI','EZU','IEUR','EWZ','ILF','EWW','INDA']
#Ishares Investment Grade, IShares core aggregate Investment grade, Short, Total, 1-5 Years, 5-10 Years, 10 Years, Gov/Credit
#defunct:

etf_spdr_indexes = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
etf_dow_components = ['MMM','AXP','AMGN','AAPL','BA','CAT','CVX','CSCO','KO','DOW','GS','HD','HON','IBM','INTC','JNJ','JPM','MCD','MRK','MSFT','NKE','PG','CRM','TRV','UNH','VZ','V','WMT','WBA','DIS']

etf_bonds = ['LQD', 'AGG', 'NEAR', 'IUSB', 'ISTB', 'IMTB', 'ILTB', 'GBF']
etf_muni_bonds = ['MUB', 'SUB', 'MEAR']

etf_treasuries = ['AGZ', 'GOVT', 'BIL', 'SHV', 'SHY', 'IEI', 'IEF', 'TLT']

crypto = ['BTC-USD','ETH','RPL','BCH','EOS','LTC']

#,'GOLDAMGBD228NLBM',
FRED_Indicators = ['CPALTT01USQ657N','PAYEMS','IRLTLT01USM156N','MABMM301USM189S','LFWA64TTUSM647S','MANMM101USA189S','MICH','UMCSENT','CSCICP03USM665S','DGS10','DTB3','DGS3MO','CASTHPI','GDPC1','CIVPART','POPTOTUSA647NWDB','MEHOINUSA672N','HOSMEDUSM052N','MORTGAGE30US','TTLHH','CSUSHPINSA','EMRATIO','CPIAUCSL','PSAVERT','LRUN64TTUSQ156S','USSTHPI','NYSTHPI','M2V','GFDEBTN','DFII10','GFDEGDQ188S','CUSR0000SEHA','ETOTALUSQ176N','ERENTUSQ176N','RECPROUSM156N','T5YIFR','BAMLHYH0A0HYM2TRIV','BAMLCC0A1AAATRIV','GVZCLS','DGS1','BAMLCC0A4BBBTRIV','VXVCLS','IC4WSA','WILLMICROCAPPR','WILLLRGCAPVAL','CFNAIDIFF','MZMSL','KCFSI','T5YIE','TOTALSA','USSLIND','AWHAETP','CES0500000003','TCU','WTB3MS','WGS3MO','TWEXB','DEXCHUS','DEXUSUK','CILACBQ158SBOG','CES4348400001','FEDFUNDS','TDSP','PERMIT','GFDEGDQ188S','CP','PRFI','DRSFRMACBS','DRCCLACBS','DRBLACBS','DALLCIACBEP','USROA','USROE','RSAHORUSQ156S','MEFAINUSA672N','COMREPUSQ159N','HDTGPDUSQ163N','POP','NROU','FGCCSAQ027S','TEDRATE', 'VIXCLS', 'NFCI','INDPRO','LES1252881600Q','CUUR0000SEHA','LEU0252918500Q','BAA10Y','BAMLC0A0CM','BAMLH0A3HYC','BOGMBASE','DCOILBRENTEU','DCOILWTICO','DFF','DGS1MO','DGS30','DGS5','FPCPITOTLZGUSA','ICSA','INTDSRUSM193N','M1','M1V','MPRIME','PPIACO','SPCS20RSA','STLFSI2','T10Y2Y','T10Y3M','TB3MS','TREAST','UNRATE','WPU0911']
#FRED_Indicators = ["CPALTT01USQ657N","PAYEMS","IRLTLT01USM156N","MABMM301USM189S","LFWA64TTUSM647S","MANMM101USA189S","MICH","UMCSENT","CSCICP03USM665S","DGS10","DTB3","DGS3MO","CASTHPI","GDPC1","CIVPART","POPTOTUSA647NWDB","MEHOINUSA672N","HOSMEDUSM052N","MORTGAGE30US","TTLHH","CSUSHPINSA","EMRATIO","CPIAUCSL","PSAVERT","LRUN64TTUSQ156S","USSTHPI","NYSTHPI","M2V","GFDEBTN","DFII10","GFDEGDQ188S","CUSR0000SEHA","ETOTALUSQ176N","ERENTUSQ176N","RECPROUSM156N","T5YIFR","BAMLHYH0A0HYM2TRIV","BAMLCC0A1AAATRIV","GVZCLS","DGS1","BAMLCC0A4BBBTRIV","VXVCLS","IC4WSA","WILLMICROCAPPR","WILLLRGCAPVAL","CFNAIDIFF","MZMSL","KCFSI","T5YIE","TOTALSA","USSLIND","AWHAETP","CES0500000003","TCU","WTB3MS","WGS3MO","TWEXB","DEXCHUS","DEXUSUK","CILACBQ158SBOG","CES4348400001","FEDFUNDS","TDSP","PERMIT","GFDEGDQ188S","CP","PRFI","DRSFRMACBS","DRCCLACBS","DRBLACBS","DALLCIACBEP","USROA","USROE","RSAHORUSQ156S","MEFAINUSA672N","COMREPUSQ159N","HDTGPDUSQ163N","POP","NROU","FGCCSAQ027S","TEDRATE", "VIXCLS", "NFCI","INDPRO","LES1252881600Q","CUUR0000SEHA","LEU0252918500Q","BAA10Y","BAMLC0A0CM","BAMLH0A3HYC","BOGMBASE","DCOILBRENTEU","DCOILWTICO","DFF","DGS1MO","DGS30","DGS5","FPCPITOTLZGUSA","GOLDAMGBD228NLBM","ICSA","INTDSRUSM193N","M1","M1V","MPRIME","PPIACO","SPCS20RSA","STLFSI2","T10Y2Y","T10Y3M","TB3MS","TREAST","UNRATE","WPU0911"]

Indexes = ['^SP500TR', 'QQQ', 'DIA', 'VTWO']

etf_indexes_and_Crypto_list = [Indexes, etf_commodities, etf_metals, etf_foreign_exchanges, etf_real_estate, etf_economies, etf_bonds, etf_muni_bonds, etf_treasuries, crypto, etf_spdr_indexes, etf_dow_components]

commodities = []
for sublist in etf_indexes_and_Crypto_list:
    for val in sublist:
        commodities.append(val)

#pool2 = concurrent.futures.ProcessPoolExecutor(cores)

completed = []
def dl(name):
    subset = yf.download(name, start=start_date, end=end_date, auto_adjust=True).iloc[:, :6].dropna(axis=0, how='any')
    subset = subset[start_date.strftime('%Y-%m-%d'):end_date.strftime('%Y-%m-%d')]
    #sleep(4)
    if len(subset) != 0:
        completed.append(name)
        return (subset)

#futures2 = [pool2.submit(dl, args) for args in commodities]
#wait(futures2, timeout=4, return_when=ALL_COMPLETED)

futures2 = []
#pd.DataFrame()
for i in commodities:
    futures2.append(dl(i))

#print(futures2.describe())


2021-05-13
2003-06-05
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***********

In [2]:

commodities_ = pd.DataFrame()

for x in range(0,len(completed)):
    values = futures2[x]
    values.index = pd.to_datetime(values.index)
    values = values.resample(frequency).mean().dropna()
    values['Symbol'] = x
    values = values.loc[~values.index.duplicated(keep='last')]
    values = values.reset_index()

    commodities_ = pd.concat([commodities_,values], axis=0)



In [3]:
commodities_pvt = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
commodities_pvt.columns = completed
commodities_pvt.to_csv("/mnt/distvol/commodities.csv")


In [4]:
#commodities_pvt.describe()

,^SP500TR,QQQ,DIA,VTWO,DBO,CORN,WEAT,SOYB,JO,SGG,BAL,COW,MOO,TAGS,KOL,IAU,SLV,PGM,JJC,PALL,JJU,IFUNX,SLX,UUP,FXE,FXY,FXB,FXA,FXC,FXF,CYB,FXS,INR,REZ,IYR,ERUS,EWG,EWU,EWJ,MCHI,EZU,IEUR,EWZ,ILF,EWW,INDA,LQD,AGG,NEAR,IUSB,ISTB,IMTB,ILTB,GBF,MUB,SUB,MEAR,AGZ,GOVT,BIL,SHV,SHY,IEI,IEF,TLT,BTC-USD,ETH,RPL,BCH,EOS,LTC,XLC,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLRE,XLK,XLU,MMM,AXP,AMGN,AAPL,BA,CAT,CVX,CSCO,KO,DOW,GS,HD,HON,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PG,CRM,TRV,UNH,VZ,V,WMT,WBA,DIS
count,73.000000,73.000000,73.000000,44.000000,58.000000,45.000000,40.000000,40.000000,14.000000,14.000000,14.000000,14.000000,56.000000,38.000000,53.000000,66.000000,61.000000,14.000000,14.000000,46.000000,14.000000,10.000000,59.000000,58.000000,63.000000,58.000000,61.000000,61.000000,61.000000,61.000000,53.000000,42.000000,46.000000,57.000000,73.000000,43.000000,73.000000,73.000000,73.000000,42.000000,73.000000,29.000000,73.000000,73.000000,73.000000,38.000000,73.000000,72.000000,31.000000,29.000000,35.000000,19.000000,47.000000,58.000000,56.000000,51.000000,26.000000,51.000000,38.000000,57.000000,58.000000,73.000000,58.000000,73.000000,73.000000,28.000000,73.000000,37.000000,73.000000,66.000000,73.000000,13.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,23.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,10.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,73.000000,69.000000,73.000000,73.000000,73.000000,54.000000,73.000000,73.000000,73.000000
mean,3271.664960,90.674370,137.217716,46.819111,19.228170,26.162511,11.141780,19.758618,37.989652,43.365496,43.775641,42.178610,48.370581,29.126810,27.227890,11.601100,18.762088,45.195434,46.100968,97.332101,43.554748,9.884263,37.449834,23.375896,122.397794,96.775479,150.347117,75.088665,84.546606,97.231715,23.943568,127.508703,39.720170,42.682693,50.852328,29.454018,20.945735,25.022603,42.258564,50.373280,30.240700,42.059302,34.505389,26.531315,40.506295,29.276614,84.746984,84.737976,47.048506,46.776065,45.965859,47.044891,50.523897,95.509805,91.668621,99.155958,48.388597,101.639080,23.623429,87.896443,104.763128,74.905540,108.197760,82.884528,89.849654,8098.800552,18.683994,159.286635,15.428551,8.615151,22.554389,54.283100,57.606284,32.270998,46.350784,17.523926,48.168758,41.021017,35.864267,31.132750,37.823111,31.575888,100.012124,60.367689,98.929924,23.737755,116.630885,73.122057,67.169003,23.148800,28.194758,47.448555,150.268052,83.204312,72.161581,102.179307,25.268208,73.690235,51.719652,82.842562,39.769733,52.530044,37.235595,61.174091,62.856075,68.182814,100.901931,30.157794,76.231785,60.323953,42.065599,62.588368
std,1779.843112,73.828346,72.695050,15.958650,9.606281,11.076829,6.227582,3.409077,4.815125,4.568887,5.926122,6.709312,13.017810,10.472304,19.578878,3.625731,6.501764,5.598018,7.674433,54.508952,5.330163,0.329279,10.603825,1.661928,13.606640,14.075133,20.646247,11.234659,9.708151,8.771134,1.302287,19.838979,6.644509,17.086804,20.084288,5.755287,6.310131,5.150708,10.076657,13.619846,7.425060,5.229780,13.447845,9.089633,13.148361,5.256347,24.035468,17.410025,1.710577,3.699992,2.596623,3.130576,11.300780,15.114151,14.067283,4.245160,1.110040,8.618946,1.702684,1.651597,2.843609,7.607890,13.872021,20.416822,31.280752,13050.360071,4.674763,597.458983,7.950971,4.571477,12.639843,10.231225,39.350910,16.387355,13.979903,6.298676,28.511993,21.561214,15.210020,4.531221,29.727222,14.998419,53.606848,30.066958,63.215735,29.872594,99.322487,43.280746,27.230849,11.324531,12.432467,8.953442,55.523663,78.265410,53.132965,33.017917,14.210187,38.891924,33.264688,60.761097,19.947871,56.488683,33.573391,28.013776,63.060883,39.731679,96.391360,14.366966,63.466266,29.267269,17.863193,44.501027
min,1304.050655,26.048521,58.224690,23.128938,6.241587,12.148095,5.280156,13.745079,33.278667,35.020730,33.472524,31.068810,21.100603,16.503651,6.483713,4.275172,10.079531,

In [5]:
def Fred_Data(name):
    temp = web.DataReader(str(name), 'fred', start_date, end_date)
    temp.index = pd.to_datetime(temp.index)
    temp = temp.resample(frequency).mean().dropna()
    return(temp)

#pool1 = concurrent.futures.ProcessPoolExecutor(cores)

#futures1 = [pool1.submit(Fred_Data, args) for args in FRED_Indicators]
#wait(futures1, timeout=None, return_when=ALL_COMPLETED)

FRED_set = []
FRED_completed = []
for i in FRED_Indicators:
    FRED_completed.append(i)
    FRED_set.append(Fred_Data(i))



In [6]:

FRED_pvt = pd.DataFrame()

for x in range(0,len(FRED_completed)):
    values = FRED_set[x]
    #values.index = pd.to_datetime(FRED_set["DATE"])
    values = values.resample(frequency).mean().dropna()
    #values['Symbol'] = x
    values = values.loc[~values.index.duplicated(keep='last')]
    #values = values.reset_index()

    FRED_pvt = pd.concat([FRED_pvt,values], axis=1)

#FRED_ = pd.pivot_table(commodities_, values='Close', index=['Date'],columns=['Symbol'])
#FRED_.to_csv("/mnt/distvol/FRED_set.csv")



['CPALTT01USQ657N',
 'PAYEMS',
 'IRLTLT01USM156N',
 'MABMM301USM189S',
 'LFWA64TTUSM647S',
 'MANMM101USA189S',
 'MICH',
 'UMCSENT',
 'CSCICP03USM665S',
 'DGS10',
 'DTB3',
 'DGS3MO',
 'CASTHPI',
 'GDPC1',
 'CIVPART',
 'POPTOTUSA647NWDB',
 'MEHOINUSA672N',
 'HOSMEDUSM052N',
 'MORTGAGE30US',
 'TTLHH',
 'CSUSHPINSA',
 'EMRATIO',
 'CPIAUCSL',
 'PSAVERT',
 'LRUN64TTUSQ156S',
 'USSTHPI',
 'NYSTHPI',
 'M2V',
 'GFDEBTN',
 'DFII10',
 'GFDEGDQ188S',
 'CUSR0000SEHA',
 'ETOTALUSQ176N',
 'ERENTUSQ176N',
 'RECPROUSM156N',
 'T5YIFR',
 'BAMLHYH0A0HYM2TRIV',
 'BAMLCC0A1AAATRIV',
 'GVZCLS',
 'DGS1',
 'BAMLCC0A4BBBTRIV',
 'VXVCLS',
 'IC4WSA',
 'WILLMICROCAPPR',
 'WILLLRGCAPVAL',
 'CFNAIDIFF',
 'MZMSL',
 'KCFSI',
 'T5YIE',
 'TOTALSA',
 'USSLIND',
 'AWHAETP',
 'CES0500000003',
 'TCU',
 'WTB3MS',
 'WGS3MO',
 'TWEXB',
 'DEXCHUS',
 'DEXUSUK',
 'CILACBQ158SBOG',
 'CES4348400001',
 'FEDFUNDS',
 'TDSP',
 'PERMIT',
 'GFDEGDQ188S',
 'CP',
 'PRFI',
 'DRSFRMACBS',
 'DRCCLACBS',
 'DRBLACBS',
 'DALLCIACBEP',
 'USROA',


In [15]:

#FRED_pvt = pd.pivot_table(FRED_, index=['DATE'])
FRED_pvt.to_csv("/mnt/distvol/FRED_pvt.csv")

#print(len(FRED_.columns))
#print(len(FRED_))
#FRED_pvt.columns = FRED_completed
#FRED_pvt.describe()

In [9]:
#FRED_pvt.set_index(FRED_pvt.index)

In [10]:
combined_set = pd.concat([FRED_pvt.set_index(FRED_pvt.index),commodities_pvt],axis=1)

In [14]:

drops = combined_set.describe().loc['count'].index[combined_set.describe().loc['count']<len(combined_set)]
print(drops)
filtered = combined_set.columns.tolist()

for d in drops:
    #print(d)
    filtered.remove(d)
#filtered.remove(drops.tolist())
#combined_set[filtered].to_csv("/mnt/distvol/combined_set.csv")
combined_set[filtered].describe().to_csv("/mnt/distvol/combined_set.csv")
#filtered


Index(['CPALTT01USQ657N', 'PAYEMS', 'IRLTLT01USM156N', 'MABMM301USM189S',
       'LFWA64TTUSM647S', 'MANMM101USA189S', 'MICH', 'UMCSENT',
       'CSCICP03USM665S', 'CASTHPI',
       ...
       'SHV', 'IEI', 'BTC-USD', 'RPL', 'EOS', 'XLC', 'XLRE', 'DOW', 'CRM',
       'V'],
      dtype='object', length=135)
